In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np

In [3]:
data = pd.read_csv("Qmerge1-FirstLastHRV.csv") 
data.head()
#print(data.DESCR)

,nsrrid,gender_x,race_x,age_s1_x,ang_date,chd_dthdt,chf_date,cvd_dthdt,mi_date,stk_date,...,ihr,NN_RR,AVNN,SDNN,VLF,LF,LF_n,HF,HF_n,LF_HF
0,200079,2,3,56,NaN,NaN,NaN,NaN,NaN,NaN,...,75.112951,0.992000,798.797,46.2118,695.6390,285.530,0.256639,827.0460,0.743361,0.345241
1,200079,2,3,56,NaN,NaN,NaN,NaN,NaN,NaN,...,70.633025,0.968839,849.461,72.6343,1234.8283,1666.850,0.507907,1614.9500,0.492093,1.032140
2,200082,1,1,40,NaN,NaN,NaN,NaN,NaN,NaN,...,59.355400,0.996610,1010.860,39.4332,739.2121,531.024,0.686206,242.8310,0.313794,2.186800
3,200082,1,1,40,NaN,NaN,NaN,NaN,NaN,NaN,...,56.666320,0.996466,1058.830,146.8230,13231.3000,3964.450,0.378345,6513.9400,0.621655,0.608610
4,200108,2,1,60,NaN,NaN,NaN,NaN,NaN,NaN,...,64.954397,0.990741,923.725,27.7213,532.0790,168.406,0.672796,81.9015,0.327204,2.056200


## Column Names description
1. nsrrid = This subject identifier was created by the National Sleep Research Resource (NSRR) team to more easily match with file downloads. Use 'pptid' to link with Biologic Specimen and Data Repository Information Coordinating Center (BioLINCC) and cohort-specific data.

2. BLUE25 = Quality of Life (QOL) (Sleep Heart Health Study Visit One (SHHS1)): Felt downhearted and blue. 
        1: All of the time
        2: Most of the time
        3: A good bit of the time
        4: Some of the time
        5: A little of time
        6: None of the time

3. ihr = Instantaneous heart rate

4. NN_RR = Ratio of consecutive normal sinus beats (NN) over all cardiac inter-beat (RR) intervals

5. AVNN = Mean of all normal sinus to normal sinus interbeat intervals (NN) 

6. SDNN = Standard deviation of all normal sinus to normal sinus interbeat (NN) intervals

7. VLF = Very low frequency power: the normal sinus to normal sinus interbeat (NN) interval spectral power between 0.003 and 0.04 Hz

8. LF = Low frequency power: the normal sinus to normal sinus interbeat (NN) interval spectral power between 0.04 and 0.15 Hz

9. HF = High frequency power: the normal sinus to normal sinus interbeat (NN) interval spectral power between 0.15 and 0.4 Hz

10. HF_n = High frequency power (normalized)

11. LF_HF = The ratio of low to high frequency power

In [4]:
heart_rate_data = data.copy()

In [5]:
heart_rate_data = heart_rate_data[['nsrrid', 'gender_x', 'race_x', 'age_s1_x', 'BLUE25', 'ql209f', 'ihr', 'NN_RR', 'AVNN', 'SDNN', 'VLF', 'LF', 'HF', 'HF_n', 'LF_HF']]

In [6]:
heart_rate_data.head()
heart_rate_data.shape[0]

972

In [7]:
# Skipping every other row, to reemove duplicate rows
heart_rate_data = heart_rate_data[::2]
heart_rate_data.head()
heart_rate_data.shape[0] # row count

486

In [8]:
# Renaming columns 
heart_rate_data = heart_rate_data.rename(index=str, columns={'gender_x': 'Gender', 'race_x': 'Race', 'age_s1_x': 'Age'})
heart_rate_data = heart_rate_data[heart_rate_data['BLUE25'] != 'NAN']
heart_rate_data.head()

/anaconda3/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,nsrrid,Gender,Race,Age,BLUE25,ql209f,ihr,NN_RR,AVNN,SDNN,VLF,LF,HF,HF_n,LF_HF
0,200079,2,3,56,5.0,6.0,75.112951,0.992000,798.797,46.21180,695.6390,285.53000,827.04600,0.743361,0.345241
2,200082,1,1,40,6.0,5.0,59.355400,0.996610,1010.860,39.43320,739.2121,531.02400,242.83100,0.313794,2.186800
4,200108,2,1,60,5.0,6.0,64.954397,0.990741,923.725,27.72130,532.0790,168.40600,81.90150,0.327204,2.056200
6,200109,2,1,71,6.0,6.0,76.865428,0.997389,780.585,8.41453,55.3561,8.87934,4.38036,0.330351,2.027080
8,200112,1,1,48,5.0,5.0,62.348286,0.996785,962.336,88.53440,3987.3524,3269.69000,292.34900,0.082073,11.184200


In [26]:
dep_data = heart_rate_data.dropna(axis=0, how='any')
dep_data.shape[0]

dep_data.loc[dep_data['BLUE25'] == 1 , 'BLUE25'] = 0
dep_data.loc[dep_data['BLUE25'] == 2 , 'BLUE25'] = 1
dep_data.loc[dep_data['BLUE25'] == 3 , 'BLUE25'] = 2
dep_data.loc[dep_data['BLUE25'] == 4 , 'BLUE25'] = 3
dep_data.loc[dep_data['BLUE25'] == 5 , 'BLUE25'] = 4
dep_data.loc[dep_data['BLUE25'] == 6 , 'BLUE25'] = 5

#dep_data.loc[dep_data['BLUE25'] < 5, 'BLUE25'] = 0
#dep_data.loc[dep_data['BLUE25'] >= 5, 'BLUE25'] = 1
dep_data.head(2)

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,nsrrid,Gender,Race,Age,BLUE25,ql209f,ihr,NN_RR,AVNN,SDNN,VLF,LF,HF,HF_n,LF_HF
0,200079,2,3,56,4.0,6.0,75.112951,0.99200,798.797,46.2118,695.6390,285.530,827.046,0.743361,0.345241
2,200082,1,1,40,5.0,5.0,59.355400,0.99661,1010.860,39.4332,739.2121,531.024,242.831,0.313794,2.186800


In [27]:
#heart_rate_data[heart_rate_data['BLUE25'] == 'NAN']
#dep_data['BLUE25'].unique()
#dep_data['BLUE25'] = dep_data['BLUE25'].apply(np.int64)

In [28]:
dep_data.dtypes
#dep_data

nsrrid      int64
Gender      int64
Race        int64
Age         int64
BLUE25    float64
ql209f    float64
ihr       float64
NN_RR     float64
AVNN      float64
SDNN      float64
VLF       float64
LF        float64
HF        float64
HF_n      float64
LF_HF     float64
dtype: object

In [29]:
heart_data = dep_data[['BLUE25', 'ihr', 'NN_RR', 'AVNN', 'SDNN', 'VLF', 'LF', 'HF', 'HF_n', 'LF_HF']]

In [30]:
X = heart_data.drop('BLUE25', axis = 1)  #X = df1.drop('ql209f', axis = 1)
y = heart_data['BLUE25']


In [31]:
y.unique()
#y.head()

array([4., 5., 3., 0., 2., 1.])

In [40]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [41]:
from keras.utils import to_categorical

# One-hot encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)
print(y_train_categorical.shape)
#y_train_categorical = y_train_categorical[1:]
y_train_categorical


(335, 6)


array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [42]:
for x in y_train_categorical:
    if x[0] >0:
        print('found one')
        break

found one


## Random Forest

In [43]:
#Day 2 activity 3
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train_categorical)
rf.score(X_train, y_train_categorical)

#rf = rf.fit(X, y)
#rf.score(X, y)

1.0

In [44]:
importances = rf.feature_importances_
importances

array([0.10379765, 0.11661619, 0.10836127, 0.11667263, 0.11273708,
       0.11635849, 0.12198443, 0.10168655, 0.10178571])

In [47]:
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_test, y_test)
rf.score(X_test, y_test)

1.0

### MLP

In [19]:
from keras.models import Sequential

model = Sequential()

In [20]:
from keras.layers import Dense
number_inputs = 9
number_hidden_nodes = 6 # twice of output or 18
model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))

Instructions for updating:
Colocations handled automatically by placer.


In [21]:
number_classes = 3 # number of outputs
model.add(Dense(units=number_classes, activation='softmax'))

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6)                 60        
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 21        
Total params: 81
Trainable params: 81
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
model.fit(
    X_train,
    y_train_categorical,
    epochs=1000,
    shuffle=True,
    verbose=2
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1000
 - 0s - loss: 14.1039 - acc: 0.1194
Epoch 2/1000
 - 0s - loss: 14.1039 - acc: 0.1224
Epoch 3/1000
 - 0s - loss: 14.1039 - acc: 0.1224
Epoch 4/1000
 - 0s - loss: 14.1038 - acc: 0.1224
Epoch 5/1000
 - 0s - loss: 14.1038 - acc: 0.1224
Epoch 6/1000
 - 0s - loss: 14.1038 - acc: 0.1194
Epoch 7/1000
 - 0s - loss: 14.1038 - acc: 0.1224
Epoch 8/1000
 - 0s - loss: 14.1038 - acc: 0.1224
Epoch 9/1000
 - 0s - loss: 14.1038 - acc: 0.1224
Epoch 10/1000
 - 0s - loss: 14.1038 - acc: 0.1224
Epoch 11/1000
 - 0s - loss: 14.1038 - acc: 0.1194
Epoch 12/1000
 - 0s - loss: 14.1037 - acc: 0.1224
Epoch 13/1000
 - 0s - loss: 14.1037 - acc: 0.1224
Epoch 14/1000
 - 0s - loss: 14.1037 - acc: 0.1224
Epoch 15/1000
 - 0s - loss: 14.1037 - acc: 0.1224
Epoch 16/1000
 - 0s - loss: 14.1037 - acc: 0.1224
Epoch 17/1000
 - 0s - loss: 14.1037 - acc: 0.1194
Epoch 18/1000
 - 0s - loss: 14.1037 - acc: 0.1224
Epoch 19/1000
 - 0s - loss: 14.1037 - acc: 0.1194
Epoch 20/10

Epoch 159/1000
 - 0s - loss: 14.1027 - acc: 0.1224
Epoch 160/1000
 - 0s - loss: 14.1027 - acc: 0.1194
Epoch 161/1000
 - 0s - loss: 14.1026 - acc: 0.1194
Epoch 162/1000
 - 0s - loss: 14.1026 - acc: 0.1224
Epoch 163/1000
 - 0s - loss: 14.1026 - acc: 0.1224
Epoch 164/1000
 - 0s - loss: 14.1026 - acc: 0.1224
Epoch 165/1000
 - 0s - loss: 14.1026 - acc: 0.1224
Epoch 166/1000
 - 0s - loss: 14.1026 - acc: 0.1224
Epoch 167/1000
 - 0s - loss: 14.1026 - acc: 0.1224
Epoch 168/1000
 - 0s - loss: 14.1026 - acc: 0.1224
Epoch 169/1000
 - 0s - loss: 14.1026 - acc: 0.1224
Epoch 170/1000
 - 0s - loss: 14.1026 - acc: 0.1224
Epoch 171/1000
 - 0s - loss: 14.1026 - acc: 0.1224
Epoch 172/1000
 - 0s - loss: 14.1026 - acc: 0.1224
Epoch 173/1000
 - 0s - loss: 14.1026 - acc: 0.1194
Epoch 174/1000
 - 0s - loss: 14.1026 - acc: 0.1224
Epoch 175/1000
 - 0s - loss: 14.1026 - acc: 0.1224
Epoch 176/1000
 - 0s - loss: 14.1026 - acc: 0.1224
Epoch 177/1000
 - 0s - loss: 14.1026 - acc: 0.1224
Epoch 178/1000
 - 0s - loss: 14

Epoch 320/1000
 - 0s - loss: 14.1021 - acc: 0.1224
Epoch 321/1000
 - 0s - loss: 14.1021 - acc: 0.1224
Epoch 322/1000
 - 0s - loss: 14.1021 - acc: 0.1224
Epoch 323/1000
 - 0s - loss: 14.1021 - acc: 0.1224
Epoch 324/1000
 - 0s - loss: 14.1021 - acc: 0.1224
Epoch 325/1000
 - 0s - loss: 14.1021 - acc: 0.1224
Epoch 326/1000
 - 0s - loss: 14.1021 - acc: 0.1224
Epoch 327/1000
 - 0s - loss: 14.1021 - acc: 0.1224
Epoch 328/1000
 - 0s - loss: 14.1021 - acc: 0.1224
Epoch 329/1000
 - 0s - loss: 14.1021 - acc: 0.1224
Epoch 330/1000
 - 0s - loss: 14.1021 - acc: 0.1224
Epoch 331/1000
 - 0s - loss: 14.1021 - acc: 0.1224
Epoch 332/1000
 - 0s - loss: 14.1021 - acc: 0.1224
Epoch 333/1000
 - 0s - loss: 14.1021 - acc: 0.1224
Epoch 334/1000
 - 0s - loss: 14.1021 - acc: 0.1224
Epoch 335/1000
 - 0s - loss: 14.1021 - acc: 0.1224
Epoch 336/1000
 - 0s - loss: 14.1021 - acc: 0.1224
Epoch 337/1000
 - 0s - loss: 14.1021 - acc: 0.1224
Epoch 338/1000
 - 0s - loss: 14.1021 - acc: 0.1224
Epoch 339/1000
 - 0s - loss: 14

Epoch 481/1000
 - 0s - loss: 14.1019 - acc: 0.1224
Epoch 482/1000
 - 0s - loss: 14.1019 - acc: 0.1224
Epoch 483/1000
 - 0s - loss: 14.1018 - acc: 0.1224
Epoch 484/1000
 - 0s - loss: 14.1018 - acc: 0.1224
Epoch 485/1000
 - 0s - loss: 14.1019 - acc: 0.1224
Epoch 486/1000
 - 0s - loss: 14.1018 - acc: 0.1224
Epoch 487/1000
 - 0s - loss: 14.1018 - acc: 0.1224
Epoch 488/1000
 - 0s - loss: 14.1018 - acc: 0.1224
Epoch 489/1000
 - 0s - loss: 14.1018 - acc: 0.1224
Epoch 490/1000
 - 0s - loss: 14.1018 - acc: 0.1224
Epoch 491/1000
 - 0s - loss: 14.1018 - acc: 0.1224
Epoch 492/1000
 - 0s - loss: 14.1018 - acc: 0.1224
Epoch 493/1000
 - 0s - loss: 14.1018 - acc: 0.1224
Epoch 494/1000
 - 0s - loss: 14.1018 - acc: 0.1224
Epoch 495/1000
 - 0s - loss: 14.1018 - acc: 0.1224
Epoch 496/1000
 - 0s - loss: 14.1018 - acc: 0.1224
Epoch 497/1000
 - 0s - loss: 14.1018 - acc: 0.1224
Epoch 498/1000
 - 0s - loss: 14.1018 - acc: 0.1224
Epoch 499/1000
 - 0s - loss: 14.1018 - acc: 0.1224
Epoch 500/1000
 - 0s - loss: 14

Epoch 642/1000
 - 0s - loss: 14.1017 - acc: 0.1224
Epoch 643/1000
 - 0s - loss: 14.1017 - acc: 0.1224
Epoch 644/1000
 - 0s - loss: 14.1017 - acc: 0.1224
Epoch 645/1000
 - 0s - loss: 14.1017 - acc: 0.1224
Epoch 646/1000
 - 0s - loss: 14.1017 - acc: 0.1224
Epoch 647/1000
 - 0s - loss: 14.1017 - acc: 0.1224
Epoch 648/1000
 - 0s - loss: 14.1017 - acc: 0.1224
Epoch 649/1000
 - 0s - loss: 14.1017 - acc: 0.1224
Epoch 650/1000
 - 0s - loss: 14.1017 - acc: 0.1224
Epoch 651/1000
 - 0s - loss: 14.1017 - acc: 0.1224
Epoch 652/1000
 - 0s - loss: 14.1017 - acc: 0.1224
Epoch 653/1000
 - 0s - loss: 14.1017 - acc: 0.1224
Epoch 654/1000
 - 0s - loss: 14.1017 - acc: 0.1224
Epoch 655/1000
 - 0s - loss: 14.1017 - acc: 0.1224
Epoch 656/1000
 - 0s - loss: 14.1017 - acc: 0.1224
Epoch 657/1000
 - 0s - loss: 14.1017 - acc: 0.1224
Epoch 658/1000
 - 0s - loss: 14.1017 - acc: 0.1224
Epoch 659/1000
 - 0s - loss: 14.1017 - acc: 0.1224
Epoch 660/1000
 - 0s - loss: 14.1017 - acc: 0.1224
Epoch 661/1000
 - 0s - loss: 14

Epoch 803/1000
 - 0s - loss: 14.1016 - acc: 0.1224
Epoch 804/1000
 - 0s - loss: 14.1016 - acc: 0.1224
Epoch 805/1000
 - 0s - loss: 14.1016 - acc: 0.1224
Epoch 806/1000
 - 0s - loss: 14.1016 - acc: 0.1224
Epoch 807/1000
 - 0s - loss: 14.1016 - acc: 0.1224
Epoch 808/1000
 - 0s - loss: 14.1016 - acc: 0.1224
Epoch 809/1000
 - 0s - loss: 14.1016 - acc: 0.1224
Epoch 810/1000
 - 0s - loss: 14.1016 - acc: 0.1224
Epoch 811/1000
 - 0s - loss: 14.1016 - acc: 0.1224
Epoch 812/1000
 - 0s - loss: 14.1016 - acc: 0.1224
Epoch 813/1000
 - 0s - loss: 14.1016 - acc: 0.1224
Epoch 814/1000
 - 0s - loss: 14.1016 - acc: 0.1224
Epoch 815/1000
 - 0s - loss: 14.1016 - acc: 0.1224
Epoch 816/1000
 - 0s - loss: 14.1016 - acc: 0.1224
Epoch 817/1000
 - 0s - loss: 14.1016 - acc: 0.1224
Epoch 818/1000
 - 0s - loss: 14.1016 - acc: 0.1224
Epoch 819/1000
 - 0s - loss: 14.1016 - acc: 0.1224
Epoch 820/1000
 - 0s - loss: 14.1016 - acc: 0.1224
Epoch 821/1000
 - 0s - loss: 14.1016 - acc: 0.1194
Epoch 822/1000
 - 0s - loss: 14

Epoch 964/1000
 - 0s - loss: 14.1015 - acc: 0.1224
Epoch 965/1000
 - 0s - loss: 14.1015 - acc: 0.1224
Epoch 966/1000
 - 0s - loss: 14.1016 - acc: 0.1224
Epoch 967/1000
 - 0s - loss: 14.1015 - acc: 0.1224
Epoch 968/1000
 - 0s - loss: 14.1015 - acc: 0.1224
Epoch 969/1000
 - 0s - loss: 14.1015 - acc: 0.1224
Epoch 970/1000
 - 0s - loss: 14.1015 - acc: 0.1224
Epoch 971/1000
 - 0s - loss: 14.1015 - acc: 0.1224
Epoch 972/1000
 - 0s - loss: 14.1015 - acc: 0.1224
Epoch 973/1000
 - 0s - loss: 14.1015 - acc: 0.1224
Epoch 974/1000
 - 0s - loss: 14.1015 - acc: 0.1224
Epoch 975/1000
 - 0s - loss: 14.1015 - acc: 0.1224
Epoch 976/1000
 - 0s - loss: 14.1015 - acc: 0.1224
Epoch 977/1000
 - 0s - loss: 14.1015 - acc: 0.1224
Epoch 978/1000
 - 0s - loss: 14.1015 - acc: 0.1224
Epoch 979/1000
 - 0s - loss: 14.1015 - acc: 0.1224
Epoch 980/1000
 - 0s - loss: 14.1015 - acc: 0.1224
Epoch 981/1000
 - 0s - loss: 14.1015 - acc: 0.1224
Epoch 982/1000
 - 0s - loss: 14.1015 - acc: 0.1224
Epoch 983/1000
 - 0s - loss: 14

In [25]:
model_loss, model_accuracy = model.evaluate(
    X_test, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 13.026815959385463, Accuracy: 0.1875
